This k-means will find the min and max of data it's self and will quit once no points are to switch clusters
But you must still provide the "k" aka number of clusters

In [129]:
from math import floor
import plotly.express as px
import pandas as pd
import random
import centroid_2D as centroid_2D
import math
import plotly.graph_objects as go


#genetrate input
data_random = []
data_cooked = []

N = 30
#random imput genarator size N  between 0 to 30
for i in range(N):
    x = random.randint(0, 30)
    y = random.randint(0, 30)
    cluster = 1
    data_random.append([x,y,cluster])

# NONrandom imput genarator size N  between 0 to 30
h1 = floor(N/2) #first half of N
h2 = N-h1  #second half of N
for i in range(h1): 
    x = random.randint(15, 30)
    y = random.randint(10, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(h2): 
    x = random.randint(0, 15)
    y = random.randint(0, 15)
    cluster = 1
    data_cooked.append([x,y, cluster])

df_rand = pd.DataFrame(data_random)
df_cooked = pd.DataFrame(data_cooked)

df_rand =  df_rand.rename(columns={0:'X', 1:'Y', 2:'Cluster'})
df_cooked = df_cooked.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})

In [130]:
#now plot Rando to see what it look like
fig_rand = px.scatter(df_rand, x='X', y='Y', color='Cluster')
fig_rand.show()

In [131]:
#now plot Cooked to see what it look like
fig_rand = px.scatter(df_cooked, x='X', y='Y', color='Cluster')
fig_rand.show()

In [132]:
def calc_distance(x,y):
    return math.sqrt((x*x) + (y*y))

def k_means(k, data):
    #create centriod
    centroids = [centroid_2D.Centroid() for _ in range(k)]

    #find the min and max in data
    x_min, x_max, y_min, y_max = 0,0,0,0
    for point in data:
        x_min = min(x_min, point[0])
        x_max = max(x_max, point[0])
        y_min = min(y_min, point[1])
        y_max = max(y_max, point[1])



    numbert = 1
    for c in centroids:
        c.cluster_number = numbert
        numbert += 1
        c.x = random.randint(x_min,x_max)
        c.y = random.randint(y_min, y_max)

    centroid_switched = True #if alreas on point switches centriod then this is true
    repetitions = 0 #we use this to make sure we are not stuck in infate loop of repetitions
    while centroid_switched and repetitions < 100:
        repetitions += 1
        centroid_switched = False

        #clearCentriods arrays
        for c in centroids:
            c.clear()  

        for point in data: #for each point find the centroid its closest to and add it to that 
            min_dist = float('inf')
            for c in centroids:
                new_dist = calc_distance(point[0]-c.x, point[1]-c.y)
                if new_dist < min_dist:
                    min_dist = new_dist
                    colosest_c = c       
            colosest_c.add_point(point[0], point[1])
            if colosest_c.cluster_number != point[2]: #this means it got to new centriod so we will wan to do at least one more iteration
                centroid_switched = True
            point[2] = colosest_c.cluster_number    #in the data change the centriod is assoside with
        
        '''
        for c in centroids:
            print(c.coordinates)
        '''

        for c in centroids:
            if len(c.coordinates) > 0:
                sum_x = 0
                sum_y = 0

                for point in c.coordinates:
                    sum_x += point[0]
                    sum_y += point[1]
                
                x_avg = sum_x/len(c.coordinates)
                y_avg = sum_y/len(c.coordinates)

                c.set_values(x_avg, y_avg)
                
    return centroids
    

After seeing the data we must pick a k to use. this is for the cooked up data set. this data has points in the top right quadtent and bottom left onlny 

In [133]:
k = 2
data = data_cooked

centroids = k_means(k, data)


In [134]:
data_cooked = pd.DataFrame(data_cooked)
data_cooked = data_cooked.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})
data_cooked["Cluster"] = data_cooked['Cluster'].astype(str)

color_map = {'1': 'blue', '2': 'red','3': 'green','4': 'brown','5': 'orange',}

fig_cooked = px.scatter(data_cooked, x='X', y='Y', color='Cluster', color_discrete_map=color_map, title=str(k)+"-Means on Fictional Data")
for c in centroids:
    num = str(c.cluster_number)
    color = color_map[num]
    fig_cooked.add_trace(go.Scatter(x=[c.x_old], y=[c.y_old], name="Centroid "+num, mode='markers', marker=dict(size=25, color=color)))
    fig_cooked.add_trace(go.Scatter(x=[c.x], y=[c.y], name="Next Spot for Centroid "+num, mode='markers', marker=dict(size=15, color='black', opacity=0.5)))


fig_cooked.show()


After seeing the data we must pick a k to use. this is for the random up data set

In [135]:
k = 5
data = data_random

centroids = k_means(k, data)

In [136]:
data_random = pd.DataFrame(data_random)
data_random = data_random.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})
data_random["Cluster"] = data_random['Cluster'].astype(str)

color_map = {'1': 'blue', '2': 'red','3': 'green','4': 'brown','5': 'orange',}

fig_rand = px.scatter(data_random, x='X', y='Y', color='Cluster', color_discrete_map=color_map, title=str(k)+"-Means on Random Data")

for c in centroids:
    num = str(c.cluster_number)
    color = color_map[num]
    fig_rand.add_trace(go.Scatter(x=[c.x_old], y=[c.y_old], name="Centroid "+num, mode='markers', marker=dict(size=25, color=color)))
    fig_rand.add_trace(go.Scatter(x=[c.x], y=[c.y], name="Next Spot for Centroid "+num, mode='markers', marker=dict(size=15, color='black', opacity=0.5)))


fig_rand.show()